In [1]:
import numpy as np
import cv2
import os
import csv
import rasterio
from PIL import Image
import torch
import torch.nn as nn
from torch.nn import Conv2d as Conv2D
import torch.nn.init as init
import torch.nn.functional as F
from torch.nn import Upsample
import random
from torch.utils.data import Dataset

from model import * 

In [2]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable

In [3]:
root_dir="/home/yylab/Research/Flood/Sen1Floods11_all/Project/Data/"
train_path=root_dir+"txt/train.txt"
val_path=root_dir+"txt/val.txt"
with open(train_path, "r") as t:
    lines_train =t.readlines()
    list_train=[i[:-1] for i in lines_train]
with open(val_path, "r") as v:
    lines_val =v.readlines()
    list_val=[i[:-1] for i in lines_val]  

In [7]:
root_dir="/home/yylab/Research/Flood/Sen1Floods11_all/Project/Data/"
data=list_train[0]
S1 = root_dir+"S1/"+data+"_S1Hand.tif"
DEM = root_dir+"DEM/"+data+"_DEM.tif"
Label=root_dir+"Label/"+data+"_LabelHand.tif"

In [6]:
list_train[0]

'Bolivia_294583'

In [10]:
S1_data=np.nan_to_num(np.array(Image.open(Label)))


In [27]:
S1_data=np.nan_to_num(tifffile.imread(S1))
DEM_data=tifffile.imread(DEM)
Label_data=np.uint8(rasterio.open(Label).read())
Label_data[Label_data==255]=2
location = np.argwhere(Label_data==2).tolist()   

In [28]:
DEM_NORM=(DEM_data - np.min(DEM_data)) / (np.max(DEM_data) - np.min(DEM_data))
S1_0_NORM = (S1_data[0] - np.min(S1_data[0])) / (np.max(S1_data[0]) - np.min(S1_data[0]))
S1_1_NORM = (S1_data[1] - np.min(S1_data[1])) / (np.max(S1_data[1]) - np.min(S1_data[1]))

In [29]:
for l in location:
    DEM_NORM[l[0],l[1],l[2]]=0
    S1_0_NORM[l[1],l[2]]=0
    S1_1_NORM[l[1],l[2]]=0

In [30]:
DEM_NORM.shape

(1, 512, 512)

In [31]:
location

[[0, 0, 0],
 [0, 0, 1],
 [0, 0, 2],
 [0, 0, 3],
 [0, 0, 4],
 [0, 0, 5],
 [0, 0, 6],
 [0, 0, 7],
 [0, 0, 8],
 [0, 0, 9],
 [0, 0, 10],
 [0, 0, 11],
 [0, 0, 12],
 [0, 0, 13],
 [0, 0, 14],
 [0, 0, 15],
 [0, 0, 16],
 [0, 0, 17],
 [0, 0, 18],
 [0, 0, 19],
 [0, 0, 20],
 [0, 0, 21],
 [0, 0, 22],
 [0, 0, 23],
 [0, 0, 24],
 [0, 0, 25],
 [0, 0, 26],
 [0, 0, 27],
 [0, 0, 28],
 [0, 0, 29],
 [0, 0, 30],
 [0, 0, 31],
 [0, 0, 32],
 [0, 0, 33],
 [0, 0, 34],
 [0, 0, 35],
 [0, 0, 36],
 [0, 0, 166],
 [0, 0, 167],
 [0, 0, 168],
 [0, 0, 169],
 [0, 0, 170],
 [0, 0, 171],
 [0, 0, 172],
 [0, 0, 173],
 [0, 0, 174],
 [0, 0, 175],
 [0, 0, 176],
 [0, 0, 177],
 [0, 0, 178],
 [0, 0, 179],
 [0, 0, 180],
 [0, 0, 181],
 [0, 0, 182],
 [0, 0, 183],
 [0, 0, 184],
 [0, 0, 185],
 [0, 0, 186],
 [0, 0, 187],
 [0, 0, 188],
 [0, 0, 189],
 [0, 0, 190],
 [0, 0, 191],
 [0, 0, 192],
 [0, 0, 193],
 [0, 0, 194],
 [0, 0, 195],
 [0, 0, 196],
 [0, 0, 197],
 [0, 0, 198],
 [0, 0, 199],
 [0, 0, 200],
 [0, 0, 201],
 [0, 0, 202],
 [0, 0, 203

In [32]:
S1_0_NORM = np.expand_dims(S1_0_NORM, axis=0)
S1_1_NORM = np.expand_dims(S1_1_NORM, axis=0)


In [37]:
S1_1_NORM.shape

(1, 512, 512)

In [38]:
S0_NORM = torch.from_numpy(S1_0_NORM)
S1_NORM = torch.from_numpy(S1_1_NORM)
DEM = torch.from_numpy(DEM_NORM)

In [49]:
i0 = MRI[:,0:512,:]
i1 = MRI[:,512:1024,:]
i2 = MRI[:,1024:1536,:]


In [50]:
i1.shape

torch.Size([1, 512, 512])

In [35]:
DEM_NORM.shape

(1, 512, 512)

In [40]:
MRI = torch.cat((S0_NORM,S1_NORM,DEM),dim=1)

In [42]:
MRI.shape

torch.Size([1, 1536, 512])

In [13]:
import tifffile

In [34]:
S1_0_NORM.shape

(1, 512, 512)

In [103]:
class SARDataset(Dataset):

    def __init__(self, data_list, preprocess_func):
        self.data_list = data_list
        self.preprocess_func = preprocess_func

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, i):
        return self.preprocess_func(self.data_list[i])

def processAndAugment(data):
    
    root_dir="/home/yylab/Research/Flood/Sen1Floods11_all/Project/Data/"
    
    S1 = root_dir+"S1/"+data+"_S1Hand.tif"
    DEM = root_dir+"DEM/"+data+"_DEM.tif"
    Label=root_dir+"Label/"+data+"_LabelHand.tif"
    
    S1_data=np.nan_to_num(tifffile.imread(S1))
    DEM_data=tifffile.imread(DEM)
    Label_data=np.uint8(rasterio.open(Label).read())
    Label_data[Label_data==255]=2
    location = np.argwhere(Label_data==2).tolist()    
    

    #S1_data=np.nan_to_num(rasterio.open(S1).read())
   # DEM_data=rasterio.open(DEM).read()
    #Label_data=np.uint8(rasterio.open(Label).read())
    #Label_data[Label_data==255]=2
    #location = np.argwhere(Label_data==2).tolist()
    
    DEM_NORM=(DEM_data - np.min(DEM_data)) / (np.max(DEM_data) - np.min(DEM_data))
    S1_0_NORM = (S1_data[0] - np.min(S1_data[0])) / (np.max(S1_data[0]) - np.min(S1_data[0]))
    S1_1_NORM = (S1_data[1] - np.min(S1_data[1])) / (np.max(S1_data[1]) - np.min(S1_data[1]))

    for l in location:
        DEM_NORM[l[0],l[1],l[2]]=0
        S1_0_NORM[l[1],l[2]]=0
        S1_1_NORM[l[1],l[2]]=0
    
    S1_0_NORM = np.expand_dims(S1_0_NORM, axis=0)
    S1_1_NORM = np.expand_dims(S1_1_NORM, axis=0)
    S0_NORM = torch.from_numpy(S1_0_NORM)
    S1_NORM = torch.from_numpy(S1_1_NORM)
    
    DEM = torch.from_numpy(DEM_NORM)#.float
    Label = torch.from_numpy(Label_data).squeeze().long()
    
    SUM = torch.cat((S0_NORM,S1_NORM,DEM),dim=1)
    
    
    
    
    
    #X_data = np.stack((S1_0_NORM, S1_1_NORM, DEM_NORM), axis=0)  
    
    x_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])])

# mask只需要转换为tensor
    y_transforms = transforms.ToTensor()
    
    return SUM, Label

In [104]:
a,b=processAndAugment(list_train[0])

In [105]:
a.shape

torch.Size([1, 1536, 512])

In [106]:
b.shape

torch.Size([512, 512])

In [70]:
    x_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])])

# mask只需要转换为tensor
    y_transforms = transforms.ToTensor()

In [91]:
train_dataset = SARDataset(list_train, processAndAugment)

In [92]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)

In [93]:
i=0
for x, y in train_dataloader:
    a=x
    b=y
x

<ipython-input-87-f83d81d88d14>:35: RuntimeWarning: invalid value encountered in true_divide
  S1_0_NORM = (S1_data[0] - np.min(S1_data[0])) / (np.max(S1_data[0]) - np.min(S1_data[0]))
<ipython-input-87-f83d81d88d14>:36: RuntimeWarning: invalid value encountered in true_divide
  S1_1_NORM = (S1_data[1] - np.min(S1_data[1])) / (np.max(S1_data[1]) - np.min(S1_data[1]))


tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.5951, 0.6525, 0.6873],
          [0.0000, 0.0000, 0.0000,  ..., 0.6228, 0.6733, 0.7013],
          [0.0000, 0.0000, 0.0000,  ..., 0.6761, 0.6898, 0.7047],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.4200, 0.4200, 0.4200],
          [0.0000, 0.0000, 0.0000,  ..., 0.4200, 0.4200, 0.4200],
          [0.0000, 0.0000, 0.0000,  ..., 0.4200, 0.4200, 0.4200]]],


        [[[0.4273, 0.4273, 0.4548,  ..., 0.2747, 0.3504, 0.3830],
          [0.4287, 0.4287, 0.4501,  ..., 0.1815, 0.2761, 0.3289],
          [0.3992, 0.3992, 0.4444,  ..., 0.2049, 0.2341, 0.2614],
          ...,
          [0.2424, 0.2273, 0.2273,  ..., 0.0758, 0.0758, 0.0758],
          [0.2273, 0.2273, 0.2273,  ..., 0.0758, 0.0758, 0.0758],
          [0.2121, 0.2273, 0.2273,  ..., 0.0909, 0.1061, 0.1061]]]],
       dtype=torch.float64)

In [94]:
x.shape

torch.Size([2, 1, 1536, 512])

In [97]:
y.shape

torch.Size([2, 1, 512, 512])

In [84]:
a=x[:,:,0:512,:]

In [85]:
a.shape

torch.Size([2, 1, 512, 512])